#Create the environment


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/ESoWC

/content/drive/My Drive/ESoWC


In [4]:
import pandas as pd
import xarray as xr

import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns

#Our class
from create_dataset.make_dataset import CustomDataset

In [5]:
fn_weather_05_19 = 'Data/05_2019_weather_and_NOX.nc'
fn_weather_07_19 = 'Data/07_2019_weather_and_NOX.nc'
fn_weather_05_20 = 'Data/05_2020_weather_and_NOX.nc'
fn_weather_06_20 = 'Data/06_2020_weather_and_NOX.nc'

In [6]:
fn_land = 'Data/land_cover_data.nc'
fn_conc = 'Data/totalcolConcentretations_featured.nc'
fn_traffic = 'Data/emissions_traffic_hourly_merged.nc'

##Land

In [7]:
land_instance = CustomDataset(fn_land)
land_instance.resample("1H")
land_fixed = land_instance.get_dataset()
land_fixed = land_fixed.drop_vars('NO emissions') #They are already in the weather dataset
land_fixed = land_fixed.transpose('latitude','longitude','time')  
land_fixed

Opening dataset at :  Data/land_cover_data.nc
Done!


<xarray.Dataset>
Dimensions:    (latitude: 33, longitude: 33, time: 697)
Coordinates:
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * time       (time) datetime64[ns] 2019-05-01 ... 2019-05-30
Data variables:
    height     (latitude, longitude, time) float64 0.0 0.0 0.0 ... nan nan nan
    built      (latitude, longitude, time) float64 0.0 0.0 0.0 ... nan nan nan

##Conc

In [8]:
conc_fidex = xr.open_dataset(fn_conc)
conc_fidex

<xarray.Dataset>
Dimensions:           (latitude: 33, longitude: 33, time: 708)
Coordinates:
  * latitude          (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude         (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * time              (time) datetime64[ns] 2019-05-01T01:00:00 ... 2019-05-3...
Data variables:
    NO_tc             (latitude, longitude, time) float64 ...
    CO2_tc            (latitude, longitude, time) float64 ...
    CH4_tc            (latitude, longitude, time) float64 ...
    NO2_tc            (latitude, longitude, time) float64 ...
    CO_tc             (latitude, longitude, time) float64 ...
    O3_tc             (latitude, longitude, time) float64 ...
    NO_tc_add_trend   (latitude, longitude, time) float64 ...
    CO2_tc_add_trend  (latitude, longitude, time) float64 ...
    CH4_tc_add_trend  (latitude, longitude, time) float64 ...
    NO2_tc_add_trend  (latitude, longitude, time) float64 ...
    CO_tc_add_trend   (latitude, longitude, time) float64 ...
    O3_tc_add_trend   (latitude, longitude, time) float64 ...

##Traffic

In [9]:
traffic_instance = CustomDataset(fn_traffic)
traffic_ds= traffic_instance.get_dataset()
traffic_ds=traffic_ds.drop_vars('emissions')
lat_bins = np.arange(43,51.25,0.25)
lon_bins = np.arange(4,12.25,0.25)
traffic_ds = traffic_ds.sortby(['latitude','longitude','hour'])  
traffic_ds = traffic_ds.interp(latitude=lat_bins, longitude=lon_bins, method="linear")
days = np.arange(1,32,1)
traffic_ds=traffic_ds.expand_dims({'Days':days})
trafic_df = traffic_ds.to_dataframe()
trafic_df = trafic_df.reset_index()
trafic_df['time'] = (pd.to_datetime(trafic_df['Days']-1,errors='ignore',
                             unit='d',origin='2019-05') +
                    pd.to_timedelta(trafic_df['hour'], unit='h'))
trafic_df=trafic_df.drop(columns=['Days', 'hour'])
trafic_df = trafic_df.set_index(['latitude','longitude','time'])
traffic_fixed = trafic_df.to_xarray()
traffic_fixed = traffic_fixed.transpose('latitude','longitude','time')  
traffic_fixed

Opening dataset at :  Data/emissions_traffic_hourly_merged.nc
Done!


<xarray.Dataset>
Dimensions:    (latitude: 33, longitude: 33, time: 744)
Coordinates:
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
  * time       (time) datetime64[ns] 2019-05-01 ... 2019-05-31T23:00:00
Data variables:
    traffic    (latitude, longitude, time) float64 nan nan nan ... nan nan nan

#05_2019

In [10]:
tot_dataset_05_19 = xr.open_dataset('Data/05_2019_dataset_complete_for_model_NOX.nc')
tot_dataset_05_19 = tot_dataset_05_19.rename({'EMISSIONS_2019':'EMISSIONS'})
tot_dataset_05_19

<xarray.Dataset>
Dimensions:            (latitude: 33, longitude: 33, time: 744)
Coordinates:
  * time               (time) datetime64[ns] 2019-05-01 ... 2019-05-31T23:00:00
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
Data variables: (12/32)
    EMISSIONS          (latitude, longitude, time) float64 ...
    u10                (latitude, longitude, time) float64 ...
    v10                (latitude, longitude, time) float64 ...
    hcc                (latitude, longitude, time) float64 ...
    lcc                (latitude, longitude, time) float64 ...
    tcc                (latitude, longitude, time) float64 ...
    ...                 ...
    CO2_tc_add_trend   (latitude, longitude, time) float64 ...
    CH4_tc_add_trend   (latitude, longitude, time) float64 ...
    NO2_tc_add_trend   (latitude, longitude, time) float64 ...
    CO_tc_add_trend    (latitude, longitude, time) float64 ...
    O3_tc_add_trend    (latitude, longitude, time) float64 ...
    traffic            (latitude, longitude, time) float64 ...

#07_2019

##Land

In [11]:
land_fixed_07_19 = land_fixed

land_df = land_fixed_07_19.to_dataframe()
land_df = land_df.reset_index()
land_df.time = land_df.time + pd.DateOffset(months=2)
land_df = land_df.set_index(['latitude','longitude','time'])
land_fixed_07_19 = land_df.to_xarray()
land_fixed_07_19

<xarray.Dataset>
Dimensions:    (latitude: 33, longitude: 33, time: 697)
Coordinates:
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
  * time       (time) datetime64[ns] 2019-07-01 ... 2019-07-30
Data variables:
    height     (latitude, longitude, time) float64 0.0 0.0 0.0 ... nan nan nan
    built      (latitude, longitude, time) float64 0.0 0.0 0.0 ... nan nan nan

##Weather

In [12]:
weather_07_19 = xr.open_dataset('Data/07_2019_weather_and_NOX_for_model.nc')
#This variable is too much correlated with the tcw
weather_fixed_07_19 = weather_07_19.drop_vars('tcwv')
weather_fixed_07_19 = weather_fixed_07_19.rename({'EMISSIONS_2019':'EMISSIONS'})
weather_fixed_07_19 = weather_fixed_07_19.transpose('latitude','longitude','time')  
weather_fixed_07_19

<xarray.Dataset>
Dimensions:            (latitude: 33, longitude: 33, time: 745)
Coordinates:
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * time               (time) datetime64[ns] 2019-07-01 ... 2019-08-01
Data variables: (12/17)
    EMISSIONS          (latitude, longitude, time) float64 ...
    u10                (latitude, longitude, time) float64 ...
    v10                (latitude, longitude, time) float64 ...
    hcc                (latitude, longitude, time) float64 ...
    lcc                (latitude, longitude, time) float64 ...
    tcc                (latitude, longitude, time) float64 ...
    ...                 ...
    rel_hum            (latitude, longitude, time) float64 ...
    tcw                (latitude, longitude, time) float64 ...
    tot_wind           (latitude, longitude, time) float64 ...
    tmp_shift_1        (latitude, longitude, time) float64 ...
    tot_wind_shift_12  (latitude, longitude, time) float64 ...
    rel_hum_shift_8    (latitude, longitude, time) float64 ...

##Conc

In [13]:
conc_fidex_07_19 = conc_fidex

conc_df = conc_fidex_07_19.to_dataframe()
conc_df = conc_df.reset_index()
conc_df.time = conc_df.time + pd.DateOffset(months=2)
conc_df = conc_df.set_index(['latitude','longitude','time'])
conc_fidex_07_19 = conc_df.to_xarray()
conc_fidex_07_19

<xarray.Dataset>
Dimensions:           (latitude: 33, longitude: 33, time: 708)
Coordinates:
  * latitude          (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude         (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * time              (time) datetime64[ns] 2019-07-01T01:00:00 ... 2019-07-3...
Data variables:
    NO_tc             (latitude, longitude, time) float64 nan 4.243e-07 ... nan
    CO2_tc            (latitude, longitude, time) float64 nan 412.1 ... nan nan
    CH4_tc            (latitude, longitude, time) float64 nan 1.872e+03 ... nan
    NO2_tc            (latitude, longitude, time) float64 nan 4.603e-06 ... nan
    CO_tc             (latitude, longitude, time) float64 nan 0.001009 ... nan
    O3_tc             (latitude, longitude, time) float64 nan 0.00792 ... nan
    NO_tc_add_trend   (latitude, longitude, time) float64 nan 1.025e-06 ... nan
    CO2_tc_add_trend  (latitude, longitude, time) float64 nan 412.6 ... nan nan
    CH4_tc_add_trend  (latitude, longitude, time) float64 nan 1.879e+03 ... nan
    NO2_tc_add_trend  (latitude, longitude, time) float64 nan 3.203e-06 ... nan
    CO_tc_add_trend   (latitude, longitude, time) float64 nan 0.001065 ... nan
    O3_tc_add_trend   (latitude, longitude, time) float64 nan 0.007345 ... nan

##Traffic

In [14]:
traffic_fixed_07_19 = traffic_fixed

traffic_df = traffic_fixed_07_19.to_dataframe()
traffic_df = traffic_df.reset_index()
traffic_df.time = traffic_df.time + pd.DateOffset(months=2)
traffic_df = traffic_df.set_index(['latitude','longitude','time'])
traffic_fixed_07_19 = traffic_df.to_xarray()
traffic_fixed_07_19

<xarray.Dataset>
Dimensions:    (latitude: 33, longitude: 33, time: 744)
Coordinates:
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
  * time       (time) datetime64[ns] 2019-07-01 ... 2019-07-31T23:00:00
Data variables:
    traffic    (latitude, longitude, time) float64 nan nan nan ... nan nan nan

##Merge

In [15]:
tot_dataset_07_19 = weather_fixed_07_19.merge(land_fixed_07_19)
tot_dataset_07_19 = tot_dataset_07_19.merge(conc_fidex_07_19)
tot_dataset_07_19 = tot_dataset_07_19.merge(traffic_fixed_07_19)

tot_dataset_07_19

<xarray.Dataset>
Dimensions:            (latitude: 33, longitude: 33, time: 745)
Coordinates:
  * time               (time) datetime64[ns] 2019-07-01 ... 2019-08-01
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
Data variables: (12/32)
    EMISSIONS          (latitude, longitude, time) float64 ...
    u10                (latitude, longitude, time) float64 ...
    v10                (latitude, longitude, time) float64 ...
    hcc                (latitude, longitude, time) float64 ...
    lcc                (latitude, longitude, time) float64 ...
    tcc                (latitude, longitude, time) float64 ...
    ...                 ...
    CO2_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    CH4_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    NO2_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    CO_tc_add_trend    (latitude, longitude, time) float64 nan nan ... nan nan
    O3_tc_add_trend    (latitude, longitude, time) float64 nan nan ... nan nan
    traffic            (latitude, longitude, time) float64 nan nan ... nan nan

#05_2020

##Land

In [16]:
land_fixed_05_20 = land_fixed

land_df = land_fixed_05_20.to_dataframe()
land_df = land_df.reset_index()
land_df.time = land_df.time + pd.DateOffset(years=1)
land_df = land_df.set_index(['latitude','longitude','time'])
land_fixed_05_20 = land_df.to_xarray()
land_fixed_05_20

<xarray.Dataset>
Dimensions:    (latitude: 33, longitude: 33, time: 697)
Coordinates:
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
  * time       (time) datetime64[ns] 2020-05-01 ... 2020-05-30
Data variables:
    height     (latitude, longitude, time) float64 0.0 0.0 0.0 ... nan nan nan
    built      (latitude, longitude, time) float64 0.0 0.0 0.0 ... nan nan nan

##Weather

In [17]:
weather_05_20 = xr.open_dataset('Data/05_2020_weather_and_NOX_for_model.nc')
#This variable is too much correlated with the tcw
weather_fixed_05_20 = weather_05_20.drop_vars('tcwv')
weather_fixed_05_20 = weather_fixed_05_20.rename({'EMISSIONS_2019':'EMISSIONS'})
weather_fixed_05_20 = weather_fixed_05_20.transpose('latitude','longitude','time')  
weather_fixed_05_20

<xarray.Dataset>
Dimensions:            (latitude: 33, longitude: 33, time: 742)
Coordinates:
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * time               (time) datetime64[ns] 2020-05-01 ... 2020-05-31T21:00:00
Data variables: (12/17)
    EMISSIONS          (latitude, longitude, time) float64 ...
    u10                (latitude, longitude, time) float64 ...
    v10                (latitude, longitude, time) float64 ...
    hcc                (latitude, longitude, time) float64 ...
    lcc                (latitude, longitude, time) float64 ...
    tcc                (latitude, longitude, time) float64 ...
    ...                 ...
    rel_hum            (latitude, longitude, time) float64 ...
    tcw                (latitude, longitude, time) float64 ...
    tot_wind           (latitude, longitude, time) float64 ...
    tmp_shift_1        (latitude, longitude, time) float64 ...
    tot_wind_shift_12  (latitude, longitude, time) float64 ...
    rel_hum_shift_8    (latitude, longitude, time) float64 ...

##Conc

In [18]:
conc_fidex_05_20 = conc_fidex

conc_df = conc_fidex_05_20.to_dataframe()
conc_df = conc_df.reset_index()
conc_df.time = conc_df.time + pd.DateOffset(years=1)
conc_df = conc_df.set_index(['latitude','longitude','time'])
conc_fidex_05_20 = conc_df.to_xarray()
conc_fidex_05_20

<xarray.Dataset>
Dimensions:           (latitude: 33, longitude: 33, time: 708)
Coordinates:
  * latitude          (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude         (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * time              (time) datetime64[ns] 2020-05-01T01:00:00 ... 2020-05-3...
Data variables:
    NO_tc             (latitude, longitude, time) float64 nan 4.243e-07 ... nan
    CO2_tc            (latitude, longitude, time) float64 nan 412.1 ... nan nan
    CH4_tc            (latitude, longitude, time) float64 nan 1.872e+03 ... nan
    NO2_tc            (latitude, longitude, time) float64 nan 4.603e-06 ... nan
    CO_tc             (latitude, longitude, time) float64 nan 0.001009 ... nan
    O3_tc             (latitude, longitude, time) float64 nan 0.00792 ... nan
    NO_tc_add_trend   (latitude, longitude, time) float64 nan 1.025e-06 ... nan
    CO2_tc_add_trend  (latitude, longitude, time) float64 nan 412.6 ... nan nan
    CH4_tc_add_trend  (latitude, longitude, time) float64 nan 1.879e+03 ... nan
    NO2_tc_add_trend  (latitude, longitude, time) float64 nan 3.203e-06 ... nan
    CO_tc_add_trend   (latitude, longitude, time) float64 nan 0.001065 ... nan
    O3_tc_add_trend   (latitude, longitude, time) float64 nan 0.007345 ... nan

##Traffic

In [19]:
traffic_fixed_05_20 = traffic_fixed

traffic_df = traffic_fixed_05_20.to_dataframe()
traffic_df = traffic_df.reset_index()
traffic_df.time = traffic_df.time + pd.DateOffset(years=1)
traffic_df = traffic_df.set_index(['latitude','longitude','time'])
traffic_fixed_05_20 = traffic_df.to_xarray()
traffic_fixed_05_20

<xarray.Dataset>
Dimensions:    (latitude: 33, longitude: 33, time: 744)
Coordinates:
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
  * time       (time) datetime64[ns] 2020-05-01 ... 2020-05-31T23:00:00
Data variables:
    traffic    (latitude, longitude, time) float64 nan nan nan ... nan nan nan

##Merge

In [20]:
tot_dataset_05_20 = weather_fixed_05_20.merge(land_fixed_05_20)
tot_dataset_05_20 = tot_dataset_05_20.merge(conc_fidex_05_20)
tot_dataset_05_20 = tot_dataset_05_20.merge(traffic_fixed_05_20)

tot_dataset_05_20

<xarray.Dataset>
Dimensions:            (latitude: 33, longitude: 33, time: 744)
Coordinates:
  * time               (time) datetime64[ns] 2020-05-01 ... 2020-05-31T23:00:00
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
Data variables: (12/32)
    EMISSIONS          (latitude, longitude, time) float64 nan 7.375e-14 ... nan
    u10                (latitude, longitude, time) float64 2.363 1.762 ... nan
    v10                (latitude, longitude, time) float64 0.125 0.09452 ... nan
    hcc                (latitude, longitude, time) float64 0.8993 0.9321 ... nan
    lcc                (latitude, longitude, time) float64 0.005554 ... nan
    tcc                (latitude, longitude, time) float64 0.8993 0.9321 ... nan
    ...                 ...
    CO2_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    CH4_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    NO2_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    CO_tc_add_trend    (latitude, longitude, time) float64 nan nan ... nan nan
    O3_tc_add_trend    (latitude, longitude, time) float64 nan nan ... nan nan
    traffic            (latitude, longitude, time) float64 nan nan ... nan nan

#Merge

In [21]:
tot_dataset = tot_dataset_05_19.merge(tot_dataset_07_19)
tot_dataset = tot_dataset.merge(tot_dataset_05_20)

tot_dataset

<xarray.Dataset>
Dimensions:            (latitude: 33, longitude: 33, time: 2233)
Coordinates:
  * time               (time) datetime64[ns] 2019-05-01 ... 2020-05-31T23:00:00
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
Data variables: (12/32)
    EMISSIONS          (latitude, longitude, time) float64 nan 1.15e-12 ... nan
    u10                (latitude, longitude, time) float64 2.632 2.961 ... nan
    v10                (latitude, longitude, time) float64 -5.72 -5.035 ... nan
    hcc                (latitude, longitude, time) float64 0.0 0.0 ... nan nan
    lcc                (latitude, longitude, time) float64 0.04311 ... nan
    tcc                (latitude, longitude, time) float64 0.04311 ... nan
    ...                 ...
    CO2_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    CH4_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    NO2_tc_add_trend   (latitude, longitude, time) float64 nan nan ... nan nan
    CO_tc_add_trend    (latitude, longitude, time) float64 nan nan ... nan nan
    O3_tc_add_trend    (latitude, longitude, time) float64 nan nan ... nan nan
    traffic            (latitude, longitude, time) float64 nan nan ... nan nan

#Save

In [22]:
tot_dataset.to_netcdf('Data/3_months_dataset_complete_for_model_NOX.nc', 'w', 'NETCDF4')